In [1]:
from sklearn.model_selection import KFold
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import pandas as pd
import torch.nn.functional as F

C:\Users\jonat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = pd.read_csv('train_data.csv')


In [4]:
indices = {}

cont = 0

for x in train_dataset['Etiqueta']:
    if x not in indices:
        indices[x] = cont
        cont+=1
        
train_dataset['Etiqueta'] = train_dataset['Etiqueta'].map(indices)
train_dataset

,Texto,Etiqueta
0,¿Puedo reservar un turno para la próxima semana?,0
1,Quisiera agendar una cita médica para el marte...,0
2,¿Están disponibles los turnos para el especial...,0
3,Necesito una cita con el odontólogo lo antes p...,0
4,¿Podría obtener un horario para una consulta d...,0
...,...,...
1052,seguro de salud de la institución,5
1053,seguro de seguro de salud aceptado por la inst...,5
1054,seguro de seguro médico que acepta la institución,5
1055,seguro de enfermedad aceptado por la clínica,5


In [18]:

# Configurar el modelo y el tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=8)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Configurar la validación cruzada K-Fold
kfold = KFold(n_splits=4, shuffle=True, random_state=42)

# Entrenamiento del modelo en cada fold
for train_index, val_index in kfold.split(train_dataset):
    train_data, val_data = train_dataset.iloc[train_index], train_dataset.iloc[val_index]
    
    
    # Procesamiento de los datos de entrenamiento
    train_encodings = tokenizer(train_dataset['Texto'].tolist(), truncation=True, padding=True, return_tensors='pt').to(device)
    train_labels = torch.tensor(train_dataset['Etiqueta'].tolist()).to(device)


    # Ajustar el modelo con los datos de entrenamiento
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)  # Ajusta el learning rate
    criterion = torch.nn.CrossEntropyLoss()  # Define la función de pérdida

    batch_size = 80  # Tamaño del batch
    num_epochs = 10  # Número de epochs

    train_loader = torch.utils.data.DataLoader(list(zip(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)), batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels) 
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
         
        print(f"Epoch {epoch+1} Loss: {epoch_loss / len(train_loader)}")

model.save_pretrained('asistente') # ruta para guardar el modelo
tokenizer.save_pretrained('asistente') # ruta para guardar el tokenizer del modelo

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 Loss: 1.9239706141608102
Epoch 2 Loss: 1.5793076838765825
Epoch 3 Loss: 1.2317375029836382
Epoch 4 Loss: 0.9114630179745811
Epoch 5 Loss: 0.6031513065099716
Epoch 6 Loss: 0.3611651893172945
Epoch 7 Loss: 0.21415184970412934
Epoch 8 Loss: 0.1305133748267378
Epoch 9 Loss: 0.09164658401693616
Epoch 10 Loss: 0.06878492422401905
Epoch 1 Loss: 0.05936245433986187
Epoch 2 Loss: 0.03958175318049533
Epoch 3 Loss: 0.022152102685400417
Epoch 4 Loss: 0.01646393264776894
Epoch 5 Loss: 0.012859796107347523
Epoch 6 Loss: 0.01262057589233986
Epoch 7 Loss: 0.015146627756101745
Epoch 8 Loss: 0.0131311495123165
Epoch 9 Loss: 0.010550599345671279
Epoch 10 Loss: 0.017245543355654393
Epoch 1 Loss: 0.005472886043467692
Epoch 2 Loss: 0.004240743865791176
Epoch 3 Loss: 0.002234991506806442
Epoch 4 Loss: 0.0017115176555567554
Epoch 5 Loss: 0.0044406307112824705
Epoch 6 Loss: 0.00962656841147691
Epoch 7 Loss: 0.0021712370882076876
Epoch 8 Loss: 0.010349072831948953
Epoch 9 Loss: 0.008592298113009227
Epoc

('asistente\\tokenizer_config.json',
 'asistente\\special_tokens_map.json',
 'asistente\\vocab.txt',
 'asistente\\added_tokens.json')

In [19]:
# Ruta donde has guardado el modelo
ruta_del_modelo = "asistente"

# Cargar el modelo y el tokenizador
model = BertForSequenceClassification.from_pretrained(ruta_del_modelo)
tokenizer = BertTokenizer.from_pretrained(ruta_del_modelo)

# Ejemplo de nueva entrada para hacer predicciones
nuevo_texto = "Quiero un turno"

# Tokenizar el texto de entrada
inputs = tokenizer(nuevo_texto, truncation=True, padding=True, return_tensors='pt')

# Obtener las predicciones del modelo
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits

# Aplicar la función softmax a los logits para obtener probabilidades
probabilities = F.softmax(predictions, dim=1)

etiqueta_idx = torch.argmax(probabilities)

print(etiqueta_idx)

tensor(0)


In [20]:
def intencion(texto):
    alm = {v: k for k, v in indices.items()}
    inputs = tokenizer(texto, truncation=True, padding=True, return_tensors='pt')

    # Obtener las predicciones del modelo
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits

    # Aplicar la función softmax a los logits para obtener probabilidades
    probabilities = F.softmax(predictions, dim=1)

    etiqueta_idx = torch.argmax(probabilities)
    
    return alm[etiqueta_idx.item()]

In [25]:
consultas = {
    'consulta_turno': [
        '¿Puedo obtener un turno para una consulta con el Dr. Smith?',
        'Quisiera programar una cita médica para el próximo lunes, ¿es posible?',
        '¿Están disponibles los turnos para pediatría mañana por la tarde?'
    ],
    'pregunta_horario': [
        '¿Cuál es el horario de atención del departamento de emergencias?',
        '¿A qué hora abre la clínica los sábados?',
        '¿Hasta qué hora están recibiendo pacientes hoy?'
    ],
    'pregunta_especialidad': [
        '¿Podrían decirme si tienen un especialista en cardiología disponible?',
        '¿Quién es el neurólogo de guardia hoy?',
        '¿Tienen médicos especializados en traumatología?'
    ],
    'consulta_resultados': [
        'Me gustaría saber si mis resultados de análisis de sangre ya están disponibles.',
        '¿Dónde puedo ver los resultados de mi radiografía?',
        '¿Los resultados de mi ecografía ya están listos?'
    ],
    'pregunta_receta': [
        '¿Puedo solicitar una receta para mi medicamento?',
        'Necesito una receta para renovar mi medicación, ¿cómo puedo obtenerla?',
        '¿El médico puede enviarme la receta por correo electrónico?'
    ],
    'consulta_seguro': [
        '¿Aceptan mi seguro médico para consultas en esta clínica?',
        '¿Qué tipos de seguros médicos son aceptados aquí?',
        '¿Pueden informarme sobre los procedimientos cubiertos por mi seguro de salud?'
    ]}


In [26]:
indices

{'consulta_turno': 0,
 'pregunta_horario': 1,
 'pregunta_especialidad': 2,
 'consulta_resultados': 3,
 'pregunta_receta': 4,
 'consulta_seguro': 5}

In [27]:
for x,y in consultas.items():
    for e in y:
        print(x,' : ','Prediccion : ',intencion(e), ' | ',e)

consulta_turno  :  Prediccion :  consulta_turno  |  ¿Puedo obtener un turno para una consulta con el Dr. Smith?
consulta_turno  :  Prediccion :  consulta_turno  |  Quisiera programar una cita médica para el próximo lunes, ¿es posible?
consulta_turno  :  Prediccion :  consulta_turno  |  ¿Están disponibles los turnos para pediatría mañana por la tarde?
pregunta_horario  :  Prediccion :  pregunta_horario  |  ¿Cuál es el horario de atención del departamento de emergencias?
pregunta_horario  :  Prediccion :  consulta_seguro  |  ¿A qué hora abre la clínica los sábados?
pregunta_horario  :  Prediccion :  pregunta_horario  |  ¿Hasta qué hora están recibiendo pacientes hoy?
pregunta_especialidad  :  Prediccion :  consulta_turno  |  ¿Podrían decirme si tienen un especialista en cardiología disponible?
pregunta_especialidad  :  Prediccion :  consulta_turno  |  ¿Quién es el neurólogo de guardia hoy?
pregunta_especialidad  :  Prediccion :  pregunta_especialidad  |  ¿Tienen médicos especializados en

In [31]:
intencion('Atienden la obra social accord?')

'consulta_turno'